設定一些東西齁
===

In [2]:
##
from transformers import BertTokenizer
from ckiptagger import WS, POS, NER
import transformers
import ckip_transformers
from ckip_transformers.nlp import CkipPosTagger
from ckip_transformers.nlp import CkipWordSegmenter
from ckip_transformers.nlp import CkipNerChunker

In [3]:
tokenizer=BertTokenizer.from_pretrained("bert-base-chinese")

In [4]:
tokens ="[unused1]"
#tokens =['[unused1]']
tokenizer.add_tokens(tokens, special_tokens=True)
tokenizer.save_pretrained('path/to/save/tokenizer')
tokenizer.special_tokens_map


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'additional_special_tokens': ['[unused1]']}

In [5]:
#ws = WS("./data")
#pos = POS("./data")
#ner = NER("./data")
CKIP_POS=CkipPosTagger(model="bert-base",device=-1)
CKIP_WS=CkipWordSegmenter(model="bert-base",device=-1)
CKIP_NER=CkipNerChunker(model="bert-base",device=-1)

文本前處理
===

正式文本
---
<span style="color:red">這邊只有先做1000筆 之後再多做然後寫入txt</span>

In [172]:
with open("語料\原始華語.txt","r",encoding="utf-8") as f:
    data=f.readlines()
formal_text=[]
for i in data:
    formal_text.append(i[:-1])
formal_text[0]

'Obama大勝美國首位黑人總統'

In [173]:
#64110
#<25 61308
len(formal_text)


64110

In [174]:
formal_text=formal_text[0:1000]

In [175]:
#ws_results = ws(data[2])
#pos_results = pos(ws_results)
#ner_results = ner(ws_results, pos_results)
formal_ws=CKIP_WS(formal_text)
formal_pos=CKIP_POS(formal_ws)
formal_ner=CKIP_NER(formal_text)

Inference: 100%|██████████| 4/4 [01:01<00:00, 15.48s/it]


In [176]:
formal_ws[0]

['Obama', '大勝', '美國', '首位', '黑人', '總統']

In [177]:
formal_ner[0]
#replace

[NerToken(word='Obama', ner='PERSON', idx=(0, 5)),
 NerToken(word='美國', ner='GPE', idx=(7, 9)),
 NerToken(word='首', ner='ORDINAL', idx=(9, 10))]

In [178]:
def filter_by_ner(sent,ner_filter):
    filter_ner_sent=[]
    for j,word in enumerate(sent):
        if word.ner in ner_filter:
            filter_ner_sent.append(word.idx)
            
    return filter_ner_sent

In [179]:
ner_filter=["PERSON"]
formal_ner_person=[filter_by_ner(sent,ner_filter) for sent in formal_ner]
formal_ner_person[0]

[(0, 5)]

In [180]:
count=0
for i in formal_ner_person:
    if i!=[]:
        text_slice=formal_text[count]
        starts, ends = zip(*i)
        start_slices = [int(start) for start in starts]
        end_slices = [int(end) for end in ends]
        person=(f"{i[0][0]}:{i[0][1]}")
        for x in range(len(start_slices)):
            #print(formal_text[count][start_slices[x]:end_slices[x]])
            formal_text[count]=formal_text[count].replace(formal_text[count][start_slices[x]:end_slices[x]],"[unused1]")
            #print(formal_text[count])
    count+=1
formal_noname=formal_text

In [181]:
formal_noname[0]

'[unused1]大勝美國首位黑人總統'

In [213]:
#數字
formal_tensor=[]
#[PAD]
formal_token=[]

for text in formal_noname:
    tensor=tokenizer.encode(text)
    if len(tensor)<=30:
        tensor_fill=tensor+ [0] * (25 - len(tensor))
        formal_tensor.append(tensor_fill)
        formal_token.append(tokenizer.decode(tensor_fill))
    else:formal_noname.remove(text)


   

In [214]:
print(formal_tensor[0])
print("\n")
print(formal_token[0])

[101, 1, 1920, 1245, 5401, 1751, 7674, 855, 7946, 782, 5244, 5186, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[CLS] [unused1] 大 勝 美 國 首 位 黑 人 總 統 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


非正式文本
---

In [205]:
import csv
informal_text=[] 
with open("Gossiping-QA-Dataset-2_0.csv", mode="r", encoding="utf-8-sig") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        r=row[1].replace(" ","")
        r=r.replace(",","")
        r=r.replace(".","")
        r=r.replace("。","")
        informal_text.append(r)
       
informal_text[0]

'質感劇本成員都差很多好嗎不要拿腎結石來污辱這群人'

In [126]:
len(informal_text)

774114

In [207]:
informal_text=informal_text[0:1000]
informal_text[0]

'質感劇本成員都差很多好嗎不要拿腎結石來污辱這群人'

In [191]:
informal_ws=CKIP_WS(informal_text)
informal_pos=CKIP_POS(informal_ws)
informal_ner=CKIP_NER(informal_text)

Inference: 100%|██████████| 4/4 [00:25<00:00,  6.49s/it]


In [208]:
ner_filter=["PERSON"]
informal_ner_person=[filter_by_ner(sent,ner_filter) for sent in informal_ner]

In [209]:
count=0
for i in informal_ner_person:
    if i!=[]:
        text_slice=informal_text[count]
        starts, ends = zip(*i)
        start_slices = [int(start) for start in starts]
        end_slices = [int(end) for end in ends]
        person=(f"{i[0][0]}:{i[0][1]}")
        for x in range(len(start_slices)):
            #print(formal_text[count][start_slices[x]:end_slices[x]])
            informal_text[count]=informal_text[count].replace(informal_text[count][start_slices[x]:end_slices[x]],"[unused1]")
            #print(formal_text[count])
    count+=1
informal_noname=informal_text

In [215]:
#數字
informal_tensor=[]
#[PAD]
informal_token=[]

for text in informal_noname:
    tensor=tokenizer.encode(text)
    if len(tensor)<=30:
        tensor_fill=tensor+ [0] * (25 - len(tensor))
        informal_tensor.append(tensor_fill)
        informal_token.append(tokenizer.decode(tensor_fill))
    else:informal_noname.remove(text)

In [217]:
print(informal_tensor[0])
print("\n")
print(informal_token[0])

[101, 6549, 2697, 1206, 3315, 2768, 1519, 6963, 2345, 2523, 1914, 1962, 1621, 679, 6206, 2897, 5575, 5178, 4767, 889, 3738, 6802, 6857, 5408, 782, 102]


[CLS] 質 感 劇 本 成 員 都 差 很 多 好 嗎 不 要 拿 腎 結 石 來 污 辱 這 群 人 [SEP]
